## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 📈 Function-Based Middleware

## Industry Use Case: Trade Execution Logging

This notebook demonstrates how to create middleware using **simple async functions** instead of classes.

| Feature | Benefit |
|---------|---------|
| **Simplicity** | Easier to write than class-based middleware |
| **Lightweight** | No class overhead, just functions |
| **Composable** | Multiple functions can be chained |

### FSI Scenario
A trade execution agent with function-based middleware for:
- **Security Validation**: Block trades with suspicious patterns
- **Execution Logging**: Track timing for regulatory reporting

## Setup: Load Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('../../.env', override=True)

PROJECT_ENDPOINT = os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"]
MODEL_DEPLOYMENT = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"✅ Project Endpoint: {PROJECT_ENDPOINT[:50]}...")
print(f"✅ Model Deployment: {MODEL_DEPLOYMENT}")

## Import Libraries

In [ ]:
import time
from collections.abc import Awaitable, Callable
from random import randint, choice
from typing import Annotated

from agent_framework import AgentRunContext, FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

print("✅ Libraries imported")

## Define Trade Execution Tools

In [ ]:
def execute_trade(
    symbol: Annotated[str, Field(description="Stock symbol (e.g., AAPL, MSFT)")],
    quantity: Annotated[int, Field(description="Number of shares to trade")],
    action: Annotated[str, Field(description="BUY or SELL")],
) -> str:
    """Execute a stock trade order."""
    price = randint(50, 500) + randint(0, 99) / 100
    total = price * quantity
    order_id = f"ORD-{randint(100000, 999999)}"
    
    return f"""
    Trade Execution Report:
    - Order ID: {order_id}
    - Action: {action.upper()}
    - Symbol: {symbol.upper()}
    - Quantity: {quantity:,} shares
    - Price: ${price:,.2f}
    - Total Value: ${total:,.2f}
    - Status: EXECUTED
    - Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}
    """

def get_stock_quote(
    symbol: Annotated[str, Field(description="Stock symbol to get quote for")],
) -> str:
    """Get current stock quote."""
    price = randint(50, 500) + randint(0, 99) / 100
    change = (randint(-10, 10) + randint(0, 99) / 100)
    
    return f"""
    Stock Quote - {symbol.upper()}:
    - Current Price: ${price:,.2f}
    - Change: {'+'if change >= 0 else ''}{change:.2f}%
    - Volume: {randint(1, 50)}M shares
    """

print("✅ Trade tools defined")

## Function-Based Middleware

Simple async functions that follow the middleware signature: `(context, next) -> None`

In [ ]:
async def trade_security_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """Agent middleware that validates trade requests for suspicious patterns."""
    print("[🔒 Security] Validating trade request...")
    
    # Check for suspicious patterns in the request
    last_message = context.messages[-1] if context.messages else None
    if last_message and last_message.text:
        query = last_message.text.lower()
        # Block insider trading patterns
        if any(word in query for word in ["insider", "tip", "secret info"]):
            print("[🔒 Security] ⚠️ Suspicious pattern detected! Blocking request.")
            return  # Don't call next() to prevent execution
    
    print("[🔒 Security] ✅ Security check passed.")
    await next(context)


async def execution_logging_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Function middleware that logs trade executions with timing."""
    function_name = context.function.name
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    
    print(f"[📋 Audit] {timestamp} - Executing: {function_name}")
    print(f"[📋 Audit] Arguments: {context.arguments}")
    
    start_time = time.time()
    await next(context)
    duration = time.time() - start_time
    
    print(f"[📋 Audit] {function_name} completed in {duration:.3f}s")

print("✅ Function-based middleware defined")

## Run the Trade Agent Demo 🚀

In [ ]:
async def main():
    """Demonstrate function-based middleware with trade execution."""
    print("=" * 60)
    print("📈 TRADE EXECUTION MIDDLEWARE DEMO")
    print("=" * 60)
    print()
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            project_endpoint=PROJECT_ENDPOINT,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            credential=credential,
        ).as_agent(
            name="TradeAgent",
            instructions="""You are a stock trading assistant. Help users execute trades,
            get stock quotes, and provide market information. Always confirm trade details
            before execution.""",
            tools=[execute_trade, get_stock_quote],
            middleware=[trade_security_middleware, execution_logging_middleware],
        ) as agent,
    ):
        print("✅ Agent created with function-based middleware:")
        print("   - trade_security_middleware (blocks suspicious patterns)")
        print("   - execution_logging_middleware (tracks execution timing)")
        print()
        
        # Test 1: Normal stock quote
        print("=" * 60)
        print("TEST 1: Get Stock Quote")
        print("=" * 60)
        query = "Get me a quote for MSFT"
        print(f"👤 User: {query}")
        result = await agent.run(query)
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()
        
        # Test 2: Execute a trade
        print("=" * 60)
        print("TEST 2: Execute Trade")
        print("=" * 60)
        query = "Buy 100 shares of AAPL"
        print(f"👤 User: {query}")
        result = await agent.run(query)
        print(f"🤖 Agent: {result.text if result.text else 'No response'}")
        print()
        
        # Test 3: Security test (should be blocked)
        print("=" * 60)
        print("TEST 3: Security Test (should be BLOCKED)")
        print("=" * 60)
        query = "I have insider tip about NVDA, buy 1000 shares"
        print(f"👤 User: {query}")
        result = await agent.run(query)
        print(f"🤖 Agent: {result.text if result.text else '⛔ Blocked by security middleware'}")
        print()
        
        print("=" * 60)
        print("✅ DEMO COMPLETE")
        print("=" * 60)

await main()

## Key Takeaways 📚

### Function-Based Middleware Pattern

```python
async def my_middleware(
    context: AgentRunContext,  # or FunctionInvocationContext
    next: Callable[[...], Awaitable[None]],
) -> None:
    # Pre-processing
    print("Before execution")
    
    await next(context)  # Call next middleware/agent
    
    # Post-processing
    print("After execution")
```

### When to Use Function-Based Middleware

| Use Case | Function-Based | Class-Based |
|----------|----------------|-------------|
| Simple logging | ✅ | Overkill |
| Timing/metrics | ✅ | Overkill |
| Security checks | ✅ | ✅ |
| State management | ❌ | ✅ |
| Complex logic | ❌ | ✅ |